In [1]:
# Import necessary tools
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
from langchain.chains.llm import LLMChain
from langchain.chains.llm_math.base import LLMMathChain
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
from langchain.callbacks import StreamlitCallbackHandler
from langchain.tools.tavily_search import TavilySearchResults

In [5]:
# load env variable
groq_api_key = os.getenv("GROQ_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [7]:
llm = ChatGroq(model='gemma2-9b-it', api_key=groq_api_key)

In [8]:
# initialize wikipedia the tools
wikipedia_wrapper = WikipediaAPIWrapper()
wikipedia_tool = Tool(
    name="wikipedia",
    func=wikipedia_wrapper.run,
    description="A tool for retrieving information from Wikipedia about a given topic."
)

In [9]:
# initialize the math toos
llm_math_chain = LLMMathChain.from_llm(llm=llm)
math_tool = Tool(
    name="llm-math",
    func=llm_math_chain.run,
    description="An LLM-powered tool for solving mathematical problems, capable of handling complex calculations."
) 

In [10]:
# Initialize the web search tools
web_search = TavilySearchResults(tavily_api_key=tavily_api_key)
web_search_tools = Tool(
    name="web-search",
    func=web_search.run,
    description="A tool for searching the internet for recent or specific information about a given topic."
) 

C:\Users\HANNAN\AppData\Local\Temp\ipykernel_5968\2986739058.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search = TavilySearchResults(tavily_api_key=tavily_api_key)


In [11]:
pormpt = ChatPromptTemplate.from_template(
       """
        You are a smart AI assistant designed to solve user questions using the following tools:
        - Wikipedia: For retrieving general knowledge, historical facts, or definitions.
        - llm-math: For solving mathematical problems or performing calculations.
        - web-search: For answering questions about recent events or information not found on Wikipedia.

        Guidelines:
        - Use the most appropriate tool based on the question type.
        - If the question involves mathematics, use llm-math and present the solution step by step.
        - For general knowledge, prefer Wikipedia.
        - For current or trending topics, use web-search.

        Your task is to solve the user's question and present the answer in a clear, point-wise format.

        Question: {question}

        Answer:
    """
)

In [12]:
# LLM chain
chain = LLMChain(llm=llm, prompt=pormpt)

C:\Users\HANNAN\AppData\Local\Temp\ipykernel_5968\2344511017.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=pormpt)


In [13]:
# Initialize reasoning tools
reasoning_tool = Tool(
    name="reasoning-tool",
    func=chain.run,
    description="A tool for answering logic-based and reasoning questions."
)

In [14]:
# Combine all tolls
tools = [wikipedia_tool, math_tool, web_search_tools, reasoning_tool]

In [15]:
# Initialize agent
assistant_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    handle_parsing_errors=True 
)

C:\Users\HANNAN\AppData\Local\Temp\ipykernel_5968\3921542958.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  assistant_agent = initialize_agent(


In [16]:
assistant_agent.invoke({"input": "what is 12 + 8"})

{'input': 'what is 12 + 8', 'output': '20'}

In [17]:
assistant_agent.invoke({"input": "A box contains 5 red balls and 3 blue balls. How many balls are there in total?"})

{'input': 'A box contains 5 red balls and 3 blue balls. How many balls are there in total?',
 'output': '8'}

In [18]:
assistant_agent.invoke({"input": "Solve: 15÷3+8×2"})

{'input': 'Solve: 15÷3+8×2', 'output': '21.0'}

In [20]:
assistant_agent.invoke({"input": "The perimeter of a rectangle is 50 cm. If the length is 15 cm, what is the width?"})

{'input': 'The perimeter of a rectangle is 50 cm. If the length is 15 cm, what is the width?',
 'output': '10 cm'}